In [2]:
import dash
from dash import dcc, html, Input, Output
import geopandas as gpd
import plotly.graph_objects as go
import rioxarray  # For raster clipping
from scipy.interpolate import griddata
import numpy as np
import xarray as xr
import pandas as pd
from shapely.geometry import Point

/Users/sidharthrai/Documents/hackathone_2024/vEnv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
# Load EC data and create GeoDataFrame
soil_analysis = pd.read_excel("../Data/soil_analysis/24 KSU TAPS Soil texture.xlsx", skiprows=1)
geometry = [Point(xy) for xy in zip(soil_analysis['Lng'], soil_analysis['Lat'])]
soil_analysis_data = gpd.GeoDataFrame(soil_analysis, geometry=geometry)
soil_analysis_data.set_crs("EPSG:4326", inplace=True)

,Plot ID,Lat,Lng,Lab_ID,Sample ID,Sample Depth (in),OMC (%),Soil Textural Class,Sand (%),Silt (%),Clay (%),geometry
0,301,39.385176,-101.066205,119912,1,0 - 12,3.3,Silty Clay Loam,17.6,54.9,27.5,POINT (-101.06621 39.38518)
1,301,39.385176,-101.066205,119913,2,12 - 24,0.9,Silty Clay Loam,18.9,53.6,27.5,POINT (-101.06621 39.38518)
2,301,39.385176,-101.066205,119914,3,24 - 36,1.0,Silty Clay Loam,18.0,53.4,28.6,POINT (-101.06621 39.38518)
3,301,39.385176,-101.066205,119915,4,36 - 48,0.9,Silt Loam,19.2,58.4,22.4,POINT (-101.06621 39.38518)
4,301,39.385176,-101.066205,119916,5,48 - 60,0.9,Silt Loam,20.5,60.9,18.6,POINT (-101.06621 39.38518)
...,...,...,...,...,...,...,...,...,...,...,...,...
95,2604,39.387034,-101.065067,120007,96,60 - 72,0.7,Silt Loam,20.3,64.8,14.9,POINT (-101.06507 39.38703)
96,2604,39.387034,-101.065067,120008,97,72 - 84,0.6,Silt Loam,19.0,66.0,15.0,POINT (-101.06507 39.38703)
97,2604,39.387034,-101.065067,120009,98,84 - 96,0.7,Silt Loam,19.0,68.5,12.5,POINT (-101.06507 39.38703)
98,2604,39.387034,-101.065067,120010,99,96-108,0.7,Silt Loam,19.1,68.4,12.5,POINT (-101.06507 39.38703)


In [4]:
soil_analysis_data['Soil Textural Class'].unique()

array(['Silty Clay Loam', 'Silt Loam', 'Clay Loam'], dtype=object)

In [5]:
# Load plot boundary shapefile
plot_boundary = gpd.read_file("../Data/plot_boundaries/Map with all plots/2024_Colby_TAPS_Harvest_Area.shx")

# Interpolate EC Shallow and Deep
points = np.array(list(zip(soil_analysis_data.geometry.x, soil_analysis_data.geometry.y)))
soil_texture = soil_analysis_data['Soil Textural Class'].values

#reclasify the values
def soil_texture_reclasify(soil_texture):
    def reclassify(value):
        if value == 'Silty Clay Loam':
            return 1
        elif value == 'Silt Loam':
            return 2
        elif value == 'Clay Loam':
            return 3
        else:
            return 0
    reclassify_vectorized = np.vectorize(reclassify)
    reclassified_data = reclassify_vectorized(soil_texture)
    return reclassified_data
soil_analysis_data['reclassified_soil_texture'] = soil_texture_reclasify(soil_texture)
reclassify_soil_texture = soil_analysis_data['reclassified_soil_texture']

In [6]:
def soil_texture_reverse_clasify(soil_texture):
    def reclassify(value):
        if value == 1:
            return 'Silty Clay Loam'
        elif value == 2:
            return 'Silt Loam'
        elif value == 3:
            return 'Clay Loam'
        else:
            return nan 
    reclassify_vectorized = np.vectorize(reclassify)
    reclassified_data = reclassify_vectorized(soil_texture)
    return reclassified_data

In [7]:
# Define grid parameters for interpolation
x_min, x_max = points[:, 0].min(), points[:, 0].max()
y_min, y_max = points[:, 1].min(), points[:, 1].max()
grid_x, grid_y = np.mgrid[x_min:x_max:200j, y_min:y_max:200j]

# Interpolated grids
grid_z = griddata(points, reclassify_soil_texture, (grid_x, grid_y), method='nearest').astype(int)
# grid_z_string = soil_texture_reverse_clasify(grid_z_values)
# Convert interpolated grids to xarray DataArrays for clipping
soil_texture_da = xr.DataArray(grid_z, dims=("y", "x"), 
                             coords={"y": np.linspace(y_min, y_max, grid_y.shape[1]), 
                                     "x": np.linspace(x_min, x_max, grid_x.shape[0])})

# Set CRS to match the plot boundary CRS
soil_texture_da.rio.set_crs("EPSG:4326")

<xarray.DataArray (y: 200, x: 200)> Size: 320kB
array([[1, 1, 1, ..., 2, 2, 2],
       [1, 1, 1, ..., 2, 2, 2],
       [1, 1, 1, ..., 2, 2, 2],
       ...,
       [2, 2, 2, ..., 2, 2, 2],
       [2, 2, 2, ..., 2, 2, 2],
       [2, 2, 2, ..., 2, 2, 2]])
Coordinates:
  * y        (y) float64 2kB 39.39 39.39 39.39 39.39 ... 39.39 39.39 39.39 39.39
  * x        (x) float64 2kB -101.1 -101.1 -101.1 ... -101.1 -101.1 -101.1

In [8]:
plot_boundary

,Name,Block_ID,TRT_ID,Plot_ID,geometry
0,Span D,1,15,2502,"POLYGON ((-101.06572 39.387, -101.06572 39.386..."
1,Span A,4,27,204,"POLYGON ((-101.06495 39.38511, -101.06495 39.3..."
2,Span A,4,30,206,"POLYGON ((-101.06417 39.38511, -101.06417 39.3..."
3,Span A,4,2,205,"POLYGON ((-101.06456 39.38511, -101.06456 39.3..."
4,Span A,2,14,203,"POLYGON ((-101.06534 39.38512, -101.06534 39.3..."
...,...,...,...,...,...
133,Span D,3,2,2704,"POLYGON ((-101.06494 39.38717, -101.06494 39.3..."
134,Span D,3,34,2705,"POLYGON ((-101.06455 39.38716, -101.06455 39.3..."
135,Span D,1,1,2703,"POLYGON ((-101.06533 39.38717, -101.06533 39.3..."
136,Span D,1,13,2702,"POLYGON ((-101.06572 39.38717, -101.06572 39.3..."


In [14]:
fig = go.Figure()

selected_plot = plot_boundary.index[0]
# Extract selected plot boundary
selected_boundary = plot_boundary.loc[[selected_plot], 'geometry']

# Clip EC shallow and deep rasters to the selected plot boundary
soil_texture_clipped = soil_texture_da.rio.clip(selected_boundary.geometry, drop=True,all_touched = True)

In [ ]:
# Define the reclassification dictionary



In [15]:
# Plot EC Shallow
fig.add_trace(go.Heatmap(
    z=soil_texture_clipped.values,
    x=soil_texture_clipped.coords['x'].values,
    y=soil_texture_clipped.coords['y'].values,
    colorscale="ylorbr",
    colorbar=dict(title="Soil Texture"),
    zsmooth="best"))

# Add plot boundary outline
boundary_x, boundary_y = list(selected_boundary.geometry.iloc[0].exterior.xy[0]), list(selected_boundary.geometry.iloc[0].exterior.xy[1])
fig.add_trace(go.Scatter(
    x=boundary_x,
    y=boundary_y,
    mode="lines",
    line=dict(color="black", width=2),
    name="Plot Boundary"))

# Update layout
fig.update_layout(
    title=f"Clipped Soil Texture for Plot {selected_plot}",
    xaxis_title="Longitude",
    yaxis_title="Latitude",
    template="plotly_white")